In [1]:
import pandas as pd
from datasets import load_dataset
from fastpunct import FastPunct
import re
from keybert import KeyBERT
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk
import spacy
import numpy as np
import itertools
import yake
import matplotlib.pyplot as plt
import math  
import sklearn.metrics
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import spacy_transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
import en_core_web_trf
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
from nltk.tree import *
from nltk import word_tokenize, pos_tag, ne_chunk
from transformers import pipeline

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Raghav\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raghav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Raghav\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Raghav\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [2]:
# Importing the dataset
df = pd.read_excel("C:/Users/Raghav/Downloads/Dataset_Q&A_DBMS.xlsx")

In [3]:
R_patterns = [
   (r'won\'t', 'will not'),
   (r'can\'t', 'can not'),
   (r'(\w+)\'m', '\g<1> am'),
   (r'(\w+)\'ll', '\g<1> will'),
   (r'(\w+)\'d like to', '\g<1> would like to'),
   (r'(\w+)n\'t', '\g<1> not'),
   (r'(\w+)\'ve', '\g<1> have'),
   (r'(\w+)\'s', '\g<1> is'),
   (r'(\w+)\'re', '\g<1> are')
]

class REReplacer(object):
   def __init__(self, patterns = R_patterns):
      self.patterns = [(re.compile(regex), repl) for (regex, repl) in R_patterns]
   def replace(self, text):
      s = text
      for (pattern, repl) in self.patterns:
         s = re.sub(pattern, repl, s)
      return s

In [4]:
rep_word = REReplacer()

def text_lower(x):
    return x.lower()
def corrector(x):
    return x.replace('. ', '.')

df['Model Answer (ChatGPT)'] = df['Model Answer (ChatGPT)'].apply(rep_word.replace)
df['Model Answer (ChatGPT)'] = df['Model Answer (ChatGPT)'].apply(rep_word.replace)

mod_ans_set = df.iloc[:, 2].values
usr_ans_set = df.iloc[:, 3].values

df['Model Answer (ChatGPT)'] = df['Model Answer (ChatGPT)'].apply(text_lower)
df['User Answer (Web)'] = df['User Answer (Web)'].apply(text_lower)

df['Model Answer (ChatGPT)'] = df['Model Answer (ChatGPT)'].apply(corrector)
df['User Answer (Web)'] = df['User Answer (Web)'].apply(corrector)

In [5]:
def replace_punct(x):
    pattern = "[^\w\d\+\*\-\\\=\s]"   
    repl = " "
    
    return re.sub(pattern, repl, x)

def remove_extra(text):
    return " ".join(text.strip().split())

df['Model Answer (ChatGPT)'] = df['Model Answer (ChatGPT)'].apply(replace_punct)
df['User Answer (Web)'] = df['User Answer (Web)'].apply(replace_punct)

df['Model Answer (ChatGPT)'] = df['Model Answer (ChatGPT)'].apply(remove_extra)
df['User Answer (Web)'] = df['User Answer (Web)'].apply(remove_extra)

In [6]:
def keybert_keywordExtractor1(text):
    kw_model = KeyBERT()
    return kw_model.extract_keywords(text, top_n = 50)
    
def keybert_keywordExtractor2(text):
    kw_model = KeyBERT()
    return kw_model.extract_keywords(text, top_n = 101)
def yake_keywordExtractor(text, language = "en", max_ngram_size = 1, deduplication_threshold = 0.9, deduplication_algo = 'seqm', windowSize = 2, numOfKeywords = 101):
    
    custom_kw_extractor = yake.KeywordExtractor(lan = language, 
                                            n = max_ngram_size, 
                                            dedupLim = deduplication_threshold, 
                                            dedupFunc = deduplication_algo, 
                                            windowsSize = windowSize, 
                                            top = numOfKeywords, 
                                            features = None)
    
    return custom_kw_extractor.extract_keywords(text)

In [7]:
df['keybert_keys_sentence1'] = df['Model Answer (ChatGPT)'].apply(keybert_keywordExtractor1)
df['keybert_keys_sentence2'] = df['User Answer (Web)'].apply(keybert_keywordExtractor2)

df['yake_keys_sentence1'] = df['Model Answer (ChatGPT)'].apply(yake_keywordExtractor)
df['yake_keys_sentence2'] = df['User Answer (Web)'].apply(yake_keywordExtractor)

In [8]:
def scoring_unit(keywords_text1, keywords_text2):
    match = 0
    total = 0
    synonym_dict = []
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    for token in keywords_text1:
        total += token[1]
    
    if(total == 0):
        return 10
    
    for var in keywords_text2:
        syn = wordnet.synsets(var[0])
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(var[0])
        syn_words = list(set(syn_words))
        #print(syn_words)
        temp = []
        wt = word_tokenize(var[0])
        pos = pos_tag(wt)[0][1]
        for i in range(0, len(syn_words)):
            checker_wt = word_tokenize(syn_words[i])
            checker_pos = pos_tag(wt)[0][1]
            if(pos == checker_pos):
                temp.append(syn_words[i])
        synonym_dict = synonym_dict + temp
    #print(synonym_dict)
    
    for token in keywords_text1:
        syn = wordnet.synsets(token[0])
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(token[0])
        syn_words = list(set(syn_words))
        if len(set(syn_words).intersection(set(synonym_dict)))!=0:
            print(token[0])
            match += token[1]
            
    return ((match*10)/total)

In [9]:
df['keybert_score'] = df[['keybert_keys_sentence1',
                                              'keybert_keys_sentence2']].apply(lambda x:scoring_unit(x.keybert_keys_sentence1, 
                                                                                                     x.keybert_keys_sentence2), 
                                                                                                        axis=1)
df['yake_score'] = df[['yake_keys_sentence1',
                                              'yake_keys_sentence2']].apply(lambda x:scoring_unit(x.yake_keys_sentence1, 
                                                                                                     x.yake_keys_sentence2), 
                                                                                                        axis=1)
                 

dbms
database
data
security
performance
redundancy
maintain
secure
enforcing
management
processing
efficiency
maintenance
operations
information
access
efficient
processes
users
improves
accuracy
ensures
improved
functions
using
sharing
query
systems
eliminates
reduces
database
mariadb
firebase
oracle
cassandra
postgresql
mongodb
10
schema
instance
database
structure
stored
memory
data
defines
blueprint
creating
maintaining
guide
used
contains
serves
data
independence
systems
ability
modify
changes
affecting
rewrite
make
database
databases
application
applications
access
data
query
management
managing
manage
retrieve
content
organize
database
dbms
databases
stored
data
management
manage
computer
software
electronically
store
collection
organized
users
used
create
ii
manipulate
allows
update
way
databases
database
storage
structure
stored
tables
data
relational
records
hierarchical
document
columns
types
rows
object
organization
physical
type
oriented
stores
advantages
application
logic

In [10]:
num_of_inputs = len(mod_ans_set)
sim_score = [0]*num_of_inputs
sbert_score = [0]*num_of_inputs
hf_score = [0] * num_of_inputs

In [11]:
def cos_sim(sent1_emb, sent2_emb):
  cos = np.dot(sent1_emb, sent2_emb)/(norm(sent1_emb)*norm(sent2_emb))
  return cos

def similarity_score(text1, text2, model):
  mod_sent = sent_tokenize(text1)
  usr_sent = sent_tokenize(text2)

  mod_emb = []
  usr_emb = []

  for sent in mod_sent:
    sent_emb = model.encode(sent)
    mod_emb.append(sent_emb)
  
  for sent in usr_sent:
    sent_emb = model.encode(sent)
    usr_emb.append(sent_emb)

  n = len(mod_sent)
  m = len(usr_sent)

  sim_ans = 0

  for i in range(0, n):
    for j in range(0, m):
      if(cos_sim(mod_emb[i], usr_emb[j]) >= 0.7):
        sim_ans += 1
        break
  
  sim_ans /= n
  return sim_ans

In [12]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

def summarize(text):
  preprocessed_text = text.strip().replace('\n', '')
  t5_input_text = 'summarize: ' + preprocessed_text

  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

  summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return summary

In [13]:
model_simcse = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large')
model_crossencoder = CrossEncoder('cross-encoder/stsb-roberta-base')
model_sbert = SentenceTransformer('stsb-mpnet-base-v2')
model_hf = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

No sentence-transformers model found with name C:\Users\Raghav/.cache\torch\sentence_transformers\princeton-nlp_sup-simcse-roberta-large. Creating a new one with MEAN pooling.


In [14]:
for i in range(0, num_of_inputs):
    mod_ans = mod_ans_set[i]
    usr_ans = usr_ans_set[i]

    usr_ans += summarize(usr_ans)
    
    sim_val = similarity_score(mod_ans, usr_ans, model_simcse)
    sim_score[i] = sim_val
    sbert_val = similarity_score(mod_ans, usr_ans, model_sbert)
    sbert_score[i] = sbert_val
    hf_val = similarity_score(mod_ans, usr_ans, model_hf)
    hf_score[i] = hf_val

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [15]:
simcse_score = sim_score
sbert_scoring = sbert_score
hf_score_normalized = hf_score
for i in range(0, num_of_inputs):
  simcse_score[i] = sim_score[i] * 10
  sbert_scoring[i] = sbert_scoring[i] * 10
  hf_score_normalized[i] = hf_score[i] * 10

In [16]:
simcse_score = np.array(simcse_score)
sbert_scoring = np.array(sbert_scoring)
hf_score_normalized = np.array(hf_score_normalized)

df['Sim_CSE Score'] = simcse_score.tolist()
df['SBERT Score'] = sbert_scoring.tolist()
df['HF_Score'] = hf_score_normalized.tolist()

In [17]:
nlp = en_core_web_trf.load()
n = df.shape[0]

df.insert(6,"Score_camembert", np.nan)
df.insert(7,"Score_NLTK", np.nan)
df.insert(8,"Score_spacy", np.nan)

In [18]:
def spacy_name(text):
  doc = nlp(text)
  named_entities = []
  for entity in doc.ents:
    named_entities.append((entity.text, entity.label_))
  return named_entities

def nltk_name(text):
  tokens = word_tokenize(text)
  pos_tags = pos_tag(tokens)
  final_pos1 =[]
  for i in pos_tags:
    if(i[1]=='NNP' or i[1]=='NNS' or i[1]=='NN'):
      final_pos1.append(i)
  named_entities = ne_chunk(final_pos1)
  leaves = named_entities.leaves()
  return leaves

def compute_score_nltk(a,b):
  user_arr = []
  modal_arr = []
    
  for i in range(0, len(a)):
    user_arr.append(a[i][0].lower())
  for i in range(0, len(b)):
    modal_arr.append(b[i][0].lower())
        
  entity_user = list(set(user_arr))
  entity_model = list(set(modal_arr))
    
  n = len(entity_model)
  n1 = len(entity_user)

  count = 0
  if(n==0):
    return 10
  for i in range(n):
    if(entity_model[i] in entity_user):
      count+=1
  count = (count/n)*10
  return count

def compute_score_spacy(user,modal):
  user_arr = []
  modal_arr = []
    
  for i in range(0, len(user)):
    if(user[i][1]!='CARDINAL'):
        user_arr.append(user[i][0].lower())
  for i in range(0, len(modal)):
    if(modal[i][1]!='CARDINAL'):
        modal_arr.append(modal[i][0].lower())
  user = list(set(user_arr))
  modal = list(set(modal_arr))
  
  n = len(modal)
  n1 = len(user)

  count = 0
  if(n==0):
    return 10
  for i in range(n):
    if(modal[i] in user):
      count+=1
  count = (count/n)*10
  return count

def regex(text):
  text2 = ''
  for i in text:
    if ((ord(i) >= 65 and ord(i) <= 90) or (ord(i) >= 97 and ord(i) <= 122) or ord(i)==32):
      text2+=i
  text2 = " ".join(text2.strip().split())
  text2 = text2.lower()
  return text2

In [19]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

def compute_score_camembert(modal, user):
    nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    modal = nlp(modal)
    user = nlp(user)
    modal_arr = []
    user_arr = []
    for i in range(0, len(modal)):
        if(modal[i]['score']>=0.9):
            modal_arr.append(modal[i]['word'].lower())
    if(len(modal_arr)==0):
        return 10
    for i in range(0, len(user)):
        user_arr.append(user[i]['word'].lower())
    modal_arr = set(modal_arr)
    user_arr = set(user_arr)
    return (len(modal_arr.intersection(user_arr))/len(modal_arr))*10

In [20]:
for i in range(df.shape[0]):
  modal = df['Model Answer (ChatGPT)'][i]
  user = df['User Answer (Web)'][i]
  score_camembert = compute_score_camembert(modal, user)
  df['Score_camembert'][i] = score_camembert
  modal1 = regex(modal)
  user1 = regex(user)
  modal_entity_nltk = nltk_name(modal1)
  user_entity_nltk = nltk_name(user1)
  score_nltk = compute_score_nltk(user_entity_nltk,modal_entity_nltk)
  df['Score_NLTK'][i] = score_nltk
  modal_entity_spacy = spacy_name(modal)
  user_entity_spacy = spacy_name(user)
  score_spacy = compute_score_spacy(user_entity_spacy,modal_entity_spacy)
  df['Score_spacy'][i] = score_spacy

C:\Users\Raghav\AppData\Local\Temp\ipykernel_1856\1471435462.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score_camembert'][i] = score_camembert
C:\Users\Raghav\AppData\Local\Temp\ipykernel_1856\1471435462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score_NLTK'][i] = score_nltk
C:\Users\Raghav\AppData\Local\Temp\ipykernel_1856\1471435462.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score_spacy'][i] =

In [21]:
df.to_csv(r'C:\Users\Raghav\Desktop\finalscores1.csv')